In [1]:
export_folder = "sp2"
filename_prefix = "sp2"

from local_vars import root_folder
import os

export_fullpath = os.path.join(root_folder, export_folder)

if not os.path.exists(export_fullpath):
    os.makedirs(export_fullpath)
    print("Created folder: " + export_fullpath)

print "Export data to: " + export_fullpath

Created folder: j:\Temp\sp2
Export data to: j:\Temp\sp2


In [2]:
fiducials_name = "F"

fiducials_node = slicer.util.getFirstNodeByName(fiducials_name, className="vtkMRMLMarkupsFiducialNode")
print "Fiducials node: " + fiducials_node.GetName()

sequence_browser_node = slicer.util.getFirstNodeByName('', className='vtkMRMLSequenceBrowserNode')
print "Sequence browser node: " + str(sequence_browser_node.GetID())

Fiducials node: F
Sequence browser node: vtkMRMLSequenceBrowserNode1


In [3]:
num_items = sequence_browser_node.GetNumberOfItems()
n = num_items
sequence_browser_node.SelectFirstItem()

print "Number of images: " + str(n)

Number of images: 3737


In [4]:
master_node = sequence_browser_node.GetMasterSequenceNode()
image_node = sequence_browser_node.GetProxyNode(master_node)
image_transform_id = image_node.GetTransformNodeID()
if image_transform_id is None:
    print "Probably forgot to transform image node!"
    raise ValueError
image_to_reference_node = slicer.util.getNode(image_transform_id)
image_data = image_node.GetImageData()

print "Ultrasound image node: " + image_node.GetName()
print "Transform node ID: " + image_transform_id

Ultrasound image node: Image_Image
Transform node ID: vtkMRMLLinearTransformNode5


In [5]:
def transform_fiducial_array_by_matrix(fiducial_array, transform_matrix):
    """
    :param fiducial_array: numpy.array((n,3))
    :param transform_matrix: vtk.vtkMatrix4x4
    :return: numpy.array((n,3))
    """
    n = fiducial_array.shape[0]
    transformed_array = np.zeros([n, 3])
    for i in range(n):
        point_src = np.append(fiducial_array[i], 1)
        point_dst = np.array(transform_matrix.MultiplyFloatPoint(point_src))
        transformed_array[i][:] = point_dst[:3]
    return transformed_array

def find_closest_point(image_extent, fiducials_Image):
    """
    :param image_extent: array(4) = [minX, maxX, minY, maxY]
    :param fiducials_Image: numpy.array((n,3))
    :param min_distance_px: float
    :return: (min_index, min_distance_px)
    """
    min_index = 0
    min_distance_px = float_info.max
    n = fiducials_Image.shape[0]
    for i in range(n):
      x = fiducials_Image[i][0]
      y = fiducials_Image[i][1]
      z = fiducials_Image[i][2]
      # Only work with fiducials within the width and height of the image
      if x < image_extent[0] or x > image_extent[1] or y < image_extent[2] or y > image_extent[3]:
        continue
      if abs(z) < min_distance_px:
        min_distance_px = abs(z)
        min_index = i

    return (min_index, min_distance_px)

In [6]:
minimum_intensity = 16
min_distance_mm = 6.0

import numpy as np
import csv
from sys import float_info

png_writer = vtk.vtkPNGWriter()
image_to_reference_matrix = vtk.vtkMatrix4x4()
reference_to_image_matrix = vtk.vtkMatrix4x4()

image_dims = image_data.GetDimensions()
image_extent = image_data.GetExtent()

csv_file_name = filename_prefix + ".csv"
csv_fullname = os.path.join(export_fullpath, csv_file_name)
csv_file = open(csv_fullname, 'wb')
csv_writer = csv.writer(csv_file, delimiter=',')
csv_writer.writerow(['filename', 'width', 'height', 'class', 'x', 'y'])

n_fiducials = fiducials_node.GetNumberOfFiducials()
fiducial_labels = [''] * n_fiducials
fiducial_coordinates_Reference = np.zeros([n_fiducials, 3])
n_output = 0
for i in range(n_fiducials):
    fiducials_node.GetNthFiducialPosition(i, fiducial_coordinates_Reference[i])
    fiducial_labels[i] = fiducials_node.GetNthFiducialLabel(i)

for i in range(n):
    image_data = image_node.GetImageData()
    
    image_array = slicer.util.arrayFromVolume(image_node)
    avg_intensity = np.average(image_array)
    if avg_intensity < minimum_intensity:
        print "Skipping image due to low intensity {:.2f}".format(avg_intensity)
        continue
    
    img_file_name = filename_prefix + "_%04d_ultrasound" % i + ".png"
    img_fullname = os.path.join(export_fullpath, img_file_name)
    
    png_writer.SetInputData(image_data)
    png_writer.SetFileName(img_fullname)
    png_writer.Update()
    png_writer.Write()
    
    image_to_reference_node.GetMatrixTransformToWorld(image_to_reference_matrix)
    reference_to_image_matrix.DeepCopy(image_to_reference_matrix)
    reference_to_image_matrix.Invert()
    
    image_to_reference_transform = vtk.vtkTransform()
    image_to_reference_transform.SetMatrix(image_to_reference_matrix)
    pixel_to_mm = image_to_reference_transform.GetScale()[0]
    
    fiducials_Image = transform_fiducial_array_by_matrix(fiducial_coordinates_Reference, reference_to_image_matrix)
    (min_index, min_distance_px) = find_closest_point(image_extent, fiducials_Image)
    p = fiducials_Image[min_index]
    p[1] = image_extent[1] - p[1]
    
    x = p[0] / image_dims[0]
    y = p[1] / image_dims[1]
    
    if (min_distance_px * pixel_to_mm) < min_distance_mm:
        csv_writer.writerow([img_file_name, str(image_dims[0]), str(image_dims[1]), fiducial_labels[min_index], x, y])
    else:
        csv_writer.writerow([img_file_name, str(image_dims[0]), str(image_dims[1]), '0', 0, 0])
    
    n_output += 1
    try:
        sequence_browser_node.SelectNextItem()
    except:
        print "End of sequence reached"
    
    # slicer.app.processEvents()

csv_file.close()

print "{} image files and annotations written".format(n_output)

3737 image files and annotations written
